In [14]:
from helper import *

In [15]:
VALID_COMBINATIONS = [
    (2, "eng", "restaurant"),
    (2, "eng", "laptop"),
    (2, "jpn", "hotel"),
    (2, "rus", "restaurant"),
    (2, "tat", "restaurant"),
    (2, "ukr", "restaurant"),
    (2, "zho", "restaurant"),
    (2, "zho", "laptop"),
    (3, "eng", "restaurant"),
    (3, "eng", "laptop"),
    (3, "jpn", "hotel"),
    (3, "rus", "restaurant"),
    (3, "tat", "restaurant"),
    (3, "ukr", "restaurant"),
    (3, "zho", "restaurant"),
    (3, "zho", "laptop"),
]

In [16]:
for comb in VALID_COMBINATIONS:
    subtask, language, domain = comb
    
    ground_truth = get_dataset(subtask, language, domain)
    print(ground_truth[0])

{'id': 'rest16_quad_dev_1', 'text': "ca n ' t wait wait for my next visit .", 'label': [{'aspect_term': 'NULL', 'opinion_term': 'NULL', 'valence': '6.75', 'arousal': '6.38'}]}
{'id': 'laptop_quad_dev_1', 'text': 'this unit is ` ` pretty ` ` and stylish , so my high school daughter was attracted to it for that reason .', 'label': [{'aspect_term': 'unit', 'opinion_term': 'pretty', 'valence': '7.12', 'arousal': '7.12'}, {'aspect_term': 'unit', 'opinion_term': 'stylish', 'valence': '7.12', 'arousal': '7.12'}]}
{'id': 'hotel_quad_train_1', 'text': 'スタッフも親切でまた近いうちに伺いますね', 'label': [{'aspect_term': 'スタッフ', 'opinion_term': '親切', 'valence': '6.33', 'arousal': '6.17'}]}
{'id': '225:3_0', 'text': 'Виды из окна - выше всяких похвал.', 'label': [{'aspect_term': 'Виды из окна', 'opinion_term': 'выше всяких похвал', 'valence': '8.3', 'arousal': '7.6'}]}
{'id': '225:3_0', 'text': 'Тәрәзә күренешләре мактаудан да биегрәк.', 'label': [{'aspect_term': 'Тәрәзә күренешләре', 'opinion_term': 'мактаудан да б

In [17]:
key_touples = {
    2: "Triplet",
    3: "Quadruplet"    
}

for comb in VALID_COMBINATIONS:
    subtask, language, domain = comb
    filepath = f"exported_predictions/subtask_{subtask}/pred_{language}_{domain}.jsonl"
    with open(filepath, "r") as f:
        predictions = f.readlines()
    print(predictions[0])

{"ID": "rest26_aste_dev_1", "Triplet": [{"Aspect": "Food", "Opinion": "great", "VA": "7.64#7.50"}, {"Aspect": "coffee", "Opinion": "great", "VA": "7.64#7.50"}]}

{"ID": "lap26_aste_dev_1", "Triplet": [{"Aspect": "perforemce", "Opinion": "Great", "VA": "7.60#7.68"}, {"Aspect": "price", "Opinion": "great", "VA": "7.61#7.69"}]}

{"ID": "hotel26_aste_dev_1", "Triplet": [{"Aspect": "室温", "Opinion": "ちょうど良く", "VA": "6.66#6.23"}, {"Aspect": "部屋", "Opinion": "広々", "VA": "6.90#6.47"}, {"Aspect": "部屋", "Opinion": "あずましく", "VA": "6.83#6.39"}, {"Aspect": "NULL", "Opinion": "満足", "VA": "7.03#6.75"}]}

{"ID": "1077:4_0", "Triplet": [{"Aspect": "еда", "Opinion": "Понравилась", "VA": "7.18#6.93"}, {"Aspect": "повар", "Opinion": "чудо", "VA": "8.47#8.26"}, {"Aspect": "блюдо от шеф-повара", "Opinion": "заказали", "VA": "6.29#5.59"}, {"Aspect": "салат", "Opinion": "заказали", "VA": "6.29#5.59"}]}

{"ID": "1077:4_0", "Triplet": [{"Aspect": "Ризык", "Opinion": "ошады", "VA": "7.05#6.66"}, {"Aspect": "пешек

In [27]:
import json

# Erstelle ein Dict für ground_truth pro Kombination
ground_truth_dict = {}
for comb in VALID_COMBINATIONS:
    subtask, language, domain = comb
    ground_truth = get_dataset(subtask, language, domain, split="dev")
    ground_truth_dict[comb] = {entry["id"]: entry["text"] for entry in ground_truth}
    

def check_aspect_in_item(aspect, opinion, text, subtask):
    if subtask == 2:
      if aspect.lower() in text.lower() or aspect == "NULL":
        return True
    if subtask == 3:
      if aspect.lower() in text.lower() or aspect == "NULL":
        if opinion.lower() in text.lower() or opinion == "NULL":
          return True
    return False

# Jetzt filtere die predictions
for comb in VALID_COMBINATIONS:
    subtask, language, domain = comb
    filepath = f"exported_predictions/subtask_{subtask}/pred_{language}_{domain}.jsonl"
    filtered_filepath = f"exported_predictions/subtask_{subtask}/pred_{language}_{domain}.jsonl"
    
    with open(filepath, "r", encoding='utf-8') as f:
        predictions = [json.loads(line.strip()) for line in f]
    
    filtered_predictions = []
    for pred in predictions:
        pred_id = pred["ID"]
        text = ground_truth_dict[comb].get(pred_id, "")
        key = key_touples[subtask]
        if key in pred:
            filtered_items = []
            for item in pred[key]:
                aspect = item["Aspect"]
                opinion = item["Opinion"]
                if check_aspect_in_item(aspect, opinion, text, subtask):  # Case insensitive check
                    filtered_items.append(item)
                else:
                    print(f"Removed invalid aspect '{aspect}' with opinion '{opinion}' for ID {pred_id} in {comb}")
                    
            pred[key] = filtered_items
        filtered_predictions.append(pred)
    
    # Schreibe gefilterte predictions
    with open(filtered_filepath, "w", encoding='utf-8') as f:
        for pred in filtered_predictions:
            f.write(json.dumps(pred, ensure_ascii=False) + "\n")
    
    print(f"Gefilterte Predictions für {comb} gespeichert in {filtered_filepath}")

Removed invalid aspect 'plating' with opinion 'beautifully' for ID rest26_aste_dev_40 in (2, 'eng', 'restaurant')
Gefilterte Predictions für (2, 'eng', 'restaurant') gespeichert in exported_predictions/subtask_2/pred_eng_restaurant.jsonl
Gefilterte Predictions für (2, 'eng', 'laptop') gespeichert in exported_predictions/subtask_2/pred_eng_laptop.jsonl
Gefilterte Predictions für (2, 'jpn', 'hotel') gespeichert in exported_predictions/subtask_2/pred_jpn_hotel.jsonl
Removed invalid aspect 'девушка хостесс' with opinion 'внимательным' for ID 1408:5_3 in (2, 'rus', 'restaurant')
Gefilterte Predictions für (2, 'rus', 'restaurant') gespeichert in exported_predictions/subtask_2/pred_rus_restaurant.jsonl
Gefilterte Predictions für (2, 'tat', 'restaurant') gespeichert in exported_predictions/subtask_2/pred_tat_restaurant.jsonl
Removed invalid aspect 'місце' with opinion 'тихому' for ID 3480:1_6 in (2, 'ukr', 'restaurant')
Gefilterte Predictions für (2, 'ukr', 'restaurant') gespeichert in exporte